In [0]:
from pyspark.sql.functions import *

In [0]:
telco_path = "sdd_dev.sohag_test.telco_customer_churn"
telco_df = spark.read.table(telco_path)
display(telco_df)

# Processing the dataset

## Handing missing values
* NULL values might be stored as "null"

In [0]:
for column in telco_df.columns:
  telco_df = telco_df.withColumn(column, when(col(column) == "null", None).otherwise(col(column)))